In [2]:
# only necesary if you get 'ModuleNotFoundError: ecdsa or base58':
! pip install ecdsa

     |████████████████████████████████| 119 kB 15.8 MB/s 


In [1]:
# https://medium.datadriveninvestor.com/is-it-hard-to-build-a-blockchain-from-scratch-23bac74e4f
# https://stackoverflow.com/questions/34451214/how-to-sign-and-verify-signature-with-ecdsa-in-python
from ecdsa import SigningKey as sk
import datetime as d
import hashlib as h

In [58]:
class Block:
    def __init__(self, index, timestamp, data, prevhash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.prevhash = prevhash
        self.hash = self.hashblock()
    
    def hashblock(self):
        block_encryption = h.sha256()
        block_encryption.update((str(self.index)+str(self.timestamp)+str(self.data)+str(self.prevhash)).encode())
        return block_encryption.hexdigest()

    @staticmethod
    def genesisblock():
        return Block(0,d.datetime.now(),"genesis block transaction", " ")

    @staticmethod
    def newblock(lastblock, data_):
        index = lastblock.index+1
        timestamp = d.datetime.now()
        hashblock = lastblock.hash
        data = data_
        return Block(index, timestamp, data, hashblock)
    
    def __repr__(self):
      return "ID #{}".format(self.index) \
      + " \\ Timestamp:{}".format(self.timestamp) \
      + " \\ Data: {}".format(self.data) \
      + " \\ Previous Hash:{}".format(self.prevhash) \
      + " \\ Hash:{}\n".format(self.hash)

In [81]:
class Blockchain:
  def __init__(self):
    self.chain = [Block.genesisblock()]

  def add_block(self, data):
    addblock = Block.newblock(self.chain[-1], data)
    self.chain.append(addblock)
  
  def view(self):
    return self.chain
  
  def verify(self):
    blockchain_ = self.chain
    for i in range(1, len(blockchain_)):
      # checks to see if prev hash actually matches prev hash
      if blockchain_[i-1].hash != blockchain_[i].prevhash:
        return False
      # checks that index and timestamp are increasing
      if blockchain_[i].index != i or blockchain_[i-1].timestamp > blockchain_[i].timestamp:
        return False
      # makes sure that block's timestamp is not in in the future
      if d.datetime.now() < blockchain_[i].timestamp:
        return False
      # makes sure genesis block has index 0
      if blockchain_[0].index != 0:
        return False
    for i, Block in enumerate(blockchain_):
      # verifies hash
      if Blockchain.hashblock2(Block) != blockchain_[i].hash:
        return False 
    return True
  
  def hashblock2(block_):
    block_encryption = h.sha256()
    block_encryption.update((str(block_.index)+str(block_.timestamp)+str(block_.data)+str(block_.prevhash)).encode())
    return block_encryption.hexdigest()
  
  @staticmethod
  def fromHex(a):
    return bytes.fromhex(a)

In [60]:
class Transaction:
  def __init__(self, SenderPublicKey, RecipientPublicKey, amount):
    """SenderPublicKey: ecdsa.keys.VerifyingKey - sender's public key, RecipientPublicKey: ecdsa.keys.VerifyingKey - recipeient's public key,"""
    """amount: int - amount to send to recipient from sender"""
    self.senderPublicKey = SenderPublicKey
    self.recipientPublicKey = RecipientPublicKey
    self.amount = amount
  
  def dataToSign(self):
    """str: returns str of data to be signed by signature"""
    return (str(self.senderPublicKey) + str(self.recipientPublicKey) + str(self.amount)).encode()
  
  def addSig(self, signature):
    """bytes, str: adds signature and hex version of signature to the transaction object"""
    self.signature = signature
    self.hexSignature = signature.hex()

  def data(self):
    """"str: data to be included on the blockchain, includes the signature"""
    return str(self.senderPublicKey.to_string().hex()) + str(self.recipientPublicKey.to_string().hex()) + str(self.amount) + str(self.hexSignature)

In [61]:
class ECDSA:
  def __init__ (self):
    """type - ecdsa.keys.VerifyingKey: private key used to calculate signatures, public key used to verify signatures."""
    self.private_key = sk.generate()
    self.public_key = self.private_key.verifying_key
  
  def signature(self, data):
    """bytes: data of arbitrary length to be signed -> bytes: fixed length signature of 48 bytes generated from private key and input data"""
    return self.private_key.sign(data)

  def verifySignature(self, signature, data):
    """bytes: fixed length signature generated from private key and input data, bytes: data of arbitrary length used to verify the signature""" 
    """-> bool: true if signature is true, false if signature is false"""
    return self.public_key.verify(signature, data)

  def hexPrivKey(self):
    """str: representing byte data of private key in hexidecimal"""
    return self.private_key.to_string().hex()
    
  def hexPubKey(self):
    """str: representing byte data of public key in hexidecimal"""
    return self.public_key.to_string().hex()